# IMPORT PACKAGES

In [3]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import vector_tools as vt
import khFunctions as khf
from datetime import timedelta

# IMPORT DATA
- Use data from the raw ".dat" file and the ".sen" file to compile a comprehensive dataset

In [5]:
### OLD METHOD OF IMPORTING DATA
### SAVING JUST IN CASE

#dat1 = vt.datfile_to_ds('ADV/DEPL_204a.dat','ADV/DEPL_204a.vhd',32)
#sen1 = vt.senfile_to_ds('ADV/DEPL_204a.sen')
#dat1.to_netcdf('ADV/adv1_dat.nc')
#sen1.to_netcdf('ADV/adv1_sen.nc')

#dat2 = vt.datfile_to_ds('ADV/DEP205.dat','ADV/DEP205.vhd', 32)
#sen2 = vt.senfile_to_ds('ADV/DEP205.sen')
#dat2.to_netcdf('ADV/adv2_dat.nc')
#sen2.to_netcdf('ADV/adv2_sen.nc')

Importing data...
 
Creating datetime for data file...
 
Creating xarray dataset...



In [2]:
#Use .dat, .vhd. and .sen files to generate a dataset that matches format used by 
#Wheeler and Giddings 2023 to be usable in their functions
datfile1 = 'ADV/DEPL_204a.dat'
vhdfile1 = 'ADV/DEPL_204a.vhd'
senfile1 = 'ADV/DEPL_204a.sen'

datfile2 = 'ADV/DEP205.dat'
vhdfile2 = 'ADV/DEP205.vhd'
senfile2 = 'ADV/DEP205.sen'

fs = 32

#Create the raw dataset 
adv1 = vt.vector_to_ds(datfile1, vhdfile1, senfile1, fs)

#Flag the raw dataset
adv1_flagged = vt.vectorFlag(adv1)

#Trim the bad times during adv deployment and recovery
adv1_cleaned = adv1_flagged.sel(dict(time=slice('2022-08-02T11:00:00.000000000', '2022-08-11T11:00:00.000000000'),
                                     time_sen=slice('2022-08-02T11:00:00.000000000', '2022-08-11T11:00:00.000000000'),
                                     time_start=slice('2022-08-02T11:00:00.000000000', '2022-08-11T11:00:00.000000000')))

#Repeat with second deployment
adv2 = vt.vector_to_ds(datfile2, vhdfile2, senfile2, fs)
adv2_flagged = vt.vectorFlag(adv2)
adv2_cleaned = adv2_flagged.sel(dict(time=slice('2022-08-15T09:00:00.000000000', '2022-08-30T11:00:00.000000000'),
                                     time_sen=slice('2022-08-15T09:00:00.000000000', '2022-08-30T11:00:00.000000000'),
                                     time_start=slice('2022-08-15T09:00:00.000000000', '2022-08-30T11:00:00.000000000')))

Importing data
Creating timelines
Creating xarray dataset
Assigning dataset attributes
Flagging data
Flagging sensor data
Importing data
Creating timelines
Creating xarray dataset
Assigning dataset attributes
Flagging data
Flagging sensor data


In [9]:
#Export usable data
adv1_cleaned.to_netcdf('ADV/adv1_all.nc')
adv2_cleaned.to_netcdf('ADV/adv2_all.nc')

In [2]:
#Import usable data
adv1 = xr.open_dataset('ADV/adv1_all.nc')
adv2 = xr.open_dataset('ADV/adv2_all.nc')

# DATA QUALITY CONTROL

In [ ]:
#Despike adv data using the expanding ellipsoid method from Wheeler & Giddings 2023

#Conditions
badSections = [] #Bad sectiions already removed during data import
reverse = False #Set variable for reversing direction after rotation if needed

#Run expanding threshold despiking algorith and clean data with bad SNR/Correlation
adv1Despiked = khf.ProcessVec(adv1,badSections,reverse)
adv2Despiked = khf.ProcessVec(adv2,badSections,reverse)

#Function flips the secondary principle velocity component for some unknown reason
#I don't know how to fix in the function, so I manually flip it here
adv1Despiked["Secondary"] = adv1Despiked.Secondary * -1
adv2Despiked["Secondary"] = adv2Despiked.Secondary * -1

#'ProcessVec' function from Wheeler & Giddings has been modified to work with my dataset
#but the core algorithm remains unchanged

In [37]:
#Export despiked data
adv1Despiked.to_netcdf('ADV/adv1_despiked.nc')
adv2Despiked.to_netcdf('ADV/adv2_despiked.nc')

In [4]:
#Import despiked data
adv1Despiked = xr.open_dataset('ADV/adv1_despiked.nc')
adv2Despiked = xr.open_dataset('ADV/adv2_despiked.nc')

In [ ]:
# Test how the data looks compared to the 

temp = xr.open_dataset('Temperature/temp_w_rho.nc')
temp1 = temp.sel(dict(time=slice('2022-08-02T11:00:00.000000000', '2022-08-11T11:00:00.000000000')))
temp2 = temp.sel(dict(time=slice('2022-08-15T08:40:00.000000000', '2022-08-30T11:00:00.000000000')))

adv1Prim_10m = adv1Despiked.Primary.resample(time='5Min').mean()
adv1EastRaw_10m = adv1Despiked.EastRaw.resample(time='5Min').mean()

adv1Sec_10m = adv1Despiked.Secondary.resample(time='5Min').mean()
adv1NorthRaw_10m = adv1Despiked.NorthRaw.resample(time='5Min').mean()

# TEMPERATURE
plt.figure(figsize = (20,16))

plt.subplot(311)
plt.plot(temp1.time, temp1.Temperature.isel(depth=0), 'r-', lw = 1)
plt.plot(temp1.time, temp1.Temperature.isel(depth=1), 'darkorange', lw = 1)
plt.plot(temp1.time, temp1.Temperature.isel(depth=2), 'b-', lw = 1)
plt.plot(temp1.time, temp1.Temperature.isel(depth=3), 'g-', lw = 1)
plt.plot(temp1.time, temp1.Temperature.isel(depth=4), 'c-', lw = 1)
plt.plot(temp1.time, temp1.Temperature.isel(depth=5), 'y-', lw = 1)
plt.plot(temp1.time, temp1.Temperature.isel(depth=6), 'k-', lw = 1)

plt.ylabel("Temperature (Celsius)", fontsize=14)
plt.margins(x=.01)
plt.title('Temperature within SWC Kelp Forest Mooring')
plt.legend(['2m','4m','6m','8m','9.1m', '9.4m', '9.7m'], loc = 'upper left')
#=================================================================================================
# Eastern Velocity
plt.subplot(312)
#plt.plot(adcp_dep1.time, adcp_dep1.East.isel(BinDist=1), '.r', label = 'ADCP-Eastern')
#plt.plot(adcp_10mroll_dep1.time, adcp_10mroll_dep1.East,'-k', label = 'ADCP-Eastern (10-min rolling)')
plt.plot(adv1Prim_10m.time, adv1Prim_10m, '.b', label = 'ADV-U (10-min average)')
plt.plot(adv1EastRaw_10m.time, adv1EastRaw_10m, '.r', label = 'ADV-East (10-min average)')
plt.ylim(-.1,.1)
plt.legend(loc = 'upper right')
plt.axhline(y=0, c='black', lw=2)
plt.margins(x=.01)
plt.ylabel('Velocity (m/s)')
plt.title('U (1m Above Seafloor)')
#=================================================================================================
# Northern Velocity
plt.subplot(313)
#plt.plot(adcp_dep1.time, adcp_dep1.North.isel(BinDist=1), '.r', label = 'ADCP-Northern')
#plt.plot(adcp_10mroll_dep1.time, adcp_10mroll_dep1.North,'-k', label = 'ADCP-Northern (10-min rolling)')
plt.plot(adv1Sec_10m.time, adv1Sec_10m, '.b', label = 'ADV-V (10-min average)')
plt.plot(adv1NorthRaw_10m.time, adv1NorthRaw_10m, '.r', label = 'ADV-North (10-min average)')
plt.ylim(-.1,.1)
plt.legend(loc = 'upper right')
plt.axhline(y=0, c='black', lw=2)
plt.margins(x=.01)
plt.ylabel('Velocity (m/s)')
plt.title('Northern Velocity (1m Above Seafloor)')
#=================================================================================================
plt.show()

In [ ]:
adv2Prim_10m = adv2Despiked.Primary.resample(time='5Min').mean()
adv2EastRaw_10m = adv2Despiked.EastRaw.resample(time='5Min').mean()

adv2Sec_10m = adv2Despiked.Secondary.resample(time='5Min').mean()
adv2NorthRaw_10m = adv2Despiked.NorthRaw.resample(time='5Min').mean()

# TEMPERATURE
plt.figure(figsize = (20,16))

plt.subplot(311)
plt.plot(temp2.time, temp2.Temperature.isel(depth=0), 'r-', lw = 1)
plt.plot(temp2.time, temp2.Temperature.isel(depth=1), 'darkorange', lw = 1)
plt.plot(temp2.time, temp2.Temperature.isel(depth=2), 'b-', lw = 1)
plt.plot(temp2.time, temp2.Temperature.isel(depth=3), 'g-', lw = 1)
plt.plot(temp2.time, temp2.Temperature.isel(depth=4), 'c-', lw = 1)
plt.plot(temp2.time, temp2.Temperature.isel(depth=5), 'y-', lw = 1)
plt.plot(temp2.time, temp2.Temperature.isel(depth=6), 'k-', lw = 1)

plt.ylabel("Temperature (Celsius)", fontsize=14)
plt.margins(x=.01)
plt.title('Temperature within SWC Kelp Forest Mooring')
plt.legend(['2m','4m','6m','8m','9.1m', '9.4m', '9.7m'], loc = 'upper left')
#=================================================================================================
# Eastern Velocity
plt.subplot(312)
#plt.plot(adcp_dep1.time, adcp_dep1.East.isel(BinDist=1), '.r', label = 'ADCP-Eastern')
#plt.plot(adcp_10mroll_dep1.time, adcp_10mroll_dep1.East,'-k', label = 'ADCP-Eastern (10-min rolling)')
plt.plot(adv2Prim_10m.time, adv2Prim_10m, '.b', label = 'ADV-U (10-min average)')
plt.plot(adv2EastRaw_10m.time, adv2EastRaw_10m, '.r', label = 'ADV-East (10-min average)')
plt.ylim(-.1,.1)
plt.legend(loc = 'upper right')
plt.axhline(y=0, c='black', lw=2)
plt.margins(x=.01)
plt.ylabel('Velocity (m/s)')
plt.title('U (1m Above Seafloor)')
#=================================================================================================
# Northern Velocity
plt.subplot(313)
#plt.plot(adcp_dep1.time, adcp_dep1.North.isel(BinDist=1), '.r', label = 'ADCP-Northern')
#plt.plot(adcp_10mroll_dep1.time, adcp_10mroll_dep1.North,'-k', label = 'ADCP-Northern (10-min rolling)')
plt.plot(adv2Sec_10m.time, adv2Sec_10m, '.b', label = 'ADV-V (10-min average)')
plt.plot(adv2NorthRaw_10m.time, adv2NorthRaw_10m, '.r', label = 'ADV-North (10-min average)')
plt.ylim(-.1,.1)
plt.legend(loc = 'upper right')
plt.axhline(y=0, c='black', lw=2)
plt.margins(x=.01)
plt.ylabel('Velocity (m/s)')
plt.title('Northern Velocity (1m Above Seafloor)')
#=================================================================================================
plt.show()

### Repairing data gaps
- Three methods
    - Full linear interpolation across all gaps
    - Partial interpolation over gaps <= 1s and averaging longer gaps with the average of the removed data
    - Partial interpolation over gaps <= 1s and patching longer gaps

In [27]:
adv1Despiked

<xarray.Dataset>
Dimensions:        (burst: 216, time: 8294400, time_sen: 259416, time_start: 216)
Coordinates:
  * time           (time) datetime64[ns] 2022-08-02T11:40:00 ... 2022-08-11T1...
  * time_sen       (time_sen) datetime64[ns] 2022-08-02T11:00:00 ... 2022-08-...
  * time_start     (time_start) datetime64[ns] 2022-08-02T11:40:00 ... 2022-0...
  * burst          (burst) int64 3 4 5 6 7 8 9 ... 212 213 214 215 216 217 218
Data variables:
    BurstCounter   (time_start) int64 ...
    NoVelSamples   (time_start) int64 ...
    ErrorCode      (time_sen) int64 ...
    StatusCode     (time_sen) int64 ...
    BatVolt        (time_sen) float64 ...
    SoundSpeed     (time_sen) float64 ...
    Heading        (time_sen) float64 ...
    Pitch          (time_sen) float64 ...
    Roll           (time_sen) float64 ...
    Temperature    (time_sen) float64 ...
    ChecksumSen    (time_sen) int64 ...
    BurstNum       (time) int64 3 3 3 3 3 3 3 3 ... 218 218 218 218 218 218 218
    East           (time) float64 ...
    North          (time) float64 ...
    Up             (time) float64 -0.007 -0.008 -0.007 ... -0.003 -0.003 -0.002
    CSPD           (time) float64 ...
    CDIR           (time) float64 ...
    Amp1           (time) int64 ...
    Amp2           (time) int64 ...
    Amp3           (time) int64 ...
    Snr1           (time) float64 ...
    Snr2           (time) float64 ...
    Snr3           (time) float64 ...
    Corr1          (time) int64 ...
    Corr2          (time) int64 ...
    Corr3          (time) int64 ...
    Pressure       (time) float64 ...
    ChecksumDat    (time) int64 ...
    DataFlag       (time) float64 ...
    SenFlag        (time_sen) float64 ...
    Depth          (time) float64 ...
    EOrig          (time) bool ...
    NOrig          (time) bool ...
    UpOrig         (time) bool True True True True True ... True True True True
    Primary        (time) float64 ...
    Secondary      (time) float64 ...
    PrimaryOrig    (time) bool ...
    SecondaryOrig  (time) bool ...
    EastRaw        (time) float64 ...
    NorthRaw       (time) float64 ...
    UpRaw          (time) float64 ...
    dEast          (burst) float64 ...
    dNorth         (burst) float64 ...
    dUp            (burst) float64 ...
    dPrimary       (burst) float64 ...
    dSecondary     (burst) float64 ...
Attributes:
    description:                                           ADV data
    lat:                                                   36.56195999999164
    lon:                                                   -121.94174126537672
    Sampling_Rate:                                         32
    coords:                                                ENU
    CorrCutoff:                                            60
    SnrCutoff:                                             10
    AngleCutoff:                                           10000
    Theta:                                                 59.59635813519468
    despike_lp_freq (hz):                                  0.05
    despike_cutoff_expansion_fraction:                     0.01
    despike_cutoff_expansion_densityChange_end_condition:  0.95

In [33]:
def fullInterpVec(data):
    ds = data.copy(deep=True)
    
    ds['EOrig'] = missingValE = xr.where((ds.EOrig==False) | (ds.East.isnull()==True), False, True)
    ds['NOrig'] = missingValN = xr.where((ds.NOrig==False) | (ds.North.isnull()==True), False, True)
    ds['UpOrig'] = missingValUp = xr.where((ds.UpOrig==False) | (ds.Up.isnull()==True), False, True)
    ds['PrimaryOrig'] = missingValPrim = xr.where((ds.PrimaryOrig==False) | (ds.Primary.isnull()==True), False, True)
    ds['SecondaryOrig'] = missingValSec = xr.where((ds.SecondaryOrig==False) | (ds.Secondary.isnull()==True), False, True)
    
    #Perform full linear interpolation of data with no temporal limits
    dsFullInterp = ds.interpolate_na(dim="time", method="linear")
    
    vt.badDataRatio(dsFullInterp, missingValE, 'dNorth')
    vt.badDataRatio(dsFullInterp, missingValN, 'dEast')
    vt.badDataRatio(dsFullInterp, missingValUp, 'dUp')
    vt.badDataRatio(dsFullInterp, missingValPrim, 'dPrimary')
    vt.badDataRatio(dsFullInterp, missingValSec, 'dSecondary')
    
    return dsFullInterp

In [7]:
adv1Int.dUp.where(adv1Int.burst.isin(76), drop=True)

<xarray.DataArray 'dUp' (burst: 1)>
array([0.00679687])
Coordinates:
  * burst    (burst) int64 76
Attributes:
    Description:  Ratio of bad data points to the total burst size for despik...

In [22]:
#Full linear interpolation of gaps
adv1Int = vt.fullInterpVec(adv1Despiked)
adv2Int = vt.fullInterpVec(adv2Despiked)

Linearly interpolating dataset
Evaluating ratio of nans leftover in the dataset
Linearly interpolating dataset
Evaluating ratio of nans leftover in the dataset


In [8]:
# Inerpolate gaps <= 1s and and patch the remaining gaps
adv1Patch = vt.patchVec(adv1Despiked)
adv2Patch = vt.patchVec(adv2Despiked)

Interpolating gaps <= 1s
Evaluating ratio of nans leftover in the dataset
Interpolating gaps <= 1s
Evaluating ratio of nans leftover in the dataset


In [17]:
adv2Int.dUp.where(adv2Int.burst.isin(76), drop=True)

<xarray.DataArray 'dUp' (burst: 1)>
array([0.00679687])
Coordinates:
  * burst    (burst) int64 76
Attributes:
    Description:  Ratio of bad data points to the total burst size for despik...

In [23]:
# Inerpolate gaps <= 1s and average the remaining gaps
#adv1IntAvg = vt.interpAvgVec(adv1Despiked)
#adv2IntAvg = vt.interpAvgVec(adv2Despiked)
adv1IntAvg = xr.open_dataset('ADV/adv1_IntAvg.nc')

ds = adv1IntAvg.copy(deep=True)

ds['EOrig'] = missingValE = xr.where((ds.EOrig==False) | (ds.East.isnull()==True), False, True)
ds['NOrig'] = missingValN = xr.where((ds.NOrig==False) | (ds.North.isnull()==True), False, True)
ds['UpOrig'] = missingValUp = xr.where((ds.UpOrig==False) | (ds.Up.isnull()==True), False, True)
ds['PrimaryOrig'] = missingValPrim = xr.where((ds.PrimaryOrig==False) | (ds.Primary.isnull()==True), False, True)
ds['SecondaryOrig'] = missingValSec = xr.where((ds.SecondaryOrig==False) | (ds.Secondary.isnull()==True), False, True)
    
ds['dNorth'] = adv1Int.dNorth
ds['dEast'] = adv1Int.dEast
ds['dUp'] = adv1Int.dUp
ds['dPrimary'] = adv1Int.dPrimary 
ds['dSecondary'] = adv1Int.dSecondary 

In [24]:
adv2IntAvg = xr.open_dataset('ADV/adv2_IntAvg.nc')

ds2 = adv2IntAvg.copy(deep=True)

ds2['EOrig'] = missingValE = xr.where((ds2.EOrig==False) | (ds2.East.isnull()==True), False, True)
ds2['NOrig'] = missingValN = xr.where((ds2.NOrig==False) | (ds2.North.isnull()==True), False, True)
ds2['UpOrig'] = missingValUp = xr.where((ds2.UpOrig==False) | (ds2.Up.isnull()==True), False, True)
ds2['PrimaryOrig'] = missingValPrim = xr.where((ds2.PrimaryOrig==False) | (ds2.Primary.isnull()==True), False, True)
ds2['SecondaryOrig'] = missingValSec = xr.where((ds2.SecondaryOrig==False) | (ds2.Secondary.isnull()==True), False, True)
    
ds2['dNorth'] = adv2Int.dNorth
ds2['dEast'] = adv2Int.dEast
ds2['dUp'] = adv2Int.dUp
ds2['dPrimary'] = adv2Int.dPrimary 
ds2['dSecondary'] = adv2Int.dSecondary 

In [28]:
adv1Int.to_netcdf('ADV/adv1_Interp.nc')
adv2Int.to_netcdf('ADV/adv2_Interp.nc')

adv1Patch.to_netcdf('ADV/adv1_Patched.nc')
adv2Patch.to_netcdf('ADV/adv2_Patched.nc')

ds.to_netcdf('ADV/adv1_IntAverage.nc')
ds2.to_netcdf('ADV/adv2_IntAverage.nc')

## If phase wrapping is present
- Convert velocities from ENU to beam by using the transformation matrix in .hdr files
    - The following code is based on a MatLab script available on NORTEK's FAQ forums:
        https://support.nortekgroup.com/hc/en-us/articles/360029820971-How-is-a-coordinate-transformation-done-
        - Most relevant information is available in the .hdr file
        - You will need the transformation matrix, as well as heading, pitch, and roll data for each sample to make the conversions
    - Once velocities have been converted, calculate the ambiguous velocity V_amb
    - Run a patch over the entire BEAM velocity dataset
        - If phase wrap is negative: newvel = oldvel + 2*V_amb
        - If phase wrap is positive: newvel = oldvel - 2*V_amb
    -Convert patched velocities back to ENU for more user-friendly data

In [52]:
# Transformation matrix located in .hdr file as 'Transformation matrix'

T = np.array([[2.7249, -1.3770, -1.3503], #Convert matrix to multidimensional numpy array
   [-0.0161, 2.3442, -2.3308],
   [0.3472, 0.3455, 0.3389]])

# Heading, pitch and roll are the angles output in the data in degrees
# Convert to radians
hh = np.pi*(adv1_flagged['Heading']-90)/180 #Creates list of hh, pp, and rr for all datapoints
pp = np.pi * (adv1_flagged['Pitch']/180)
rr = np.pi * (adv1_flagged['Roll']/180)

# Generate empty arrays to be populated by resulting beam velocities
beam1 = np.empty(len(adv1_flagged)) # Already created to be the length of the dataset to save processing time
beam2 = np.empty(len(adv1_flagged))
beam3 = np.empty(len(adv1_flagged))

In [ ]:
# Calculate heading matrix and tilt matrix for each data point and convert ENU velocities to beam velocities

# for loop iterates for each data point
for i in range(0,len(adv1_flagged)):
    if i % 1000000 == 0: # Progress check every 1000000 rows
                print('Currently on row:', i)
            
    H = np.array([[np.cos(hh[i]), np.sin(hh[i]), 0], # Makes the heading matrix for row i
                  [-np.sin(hh[i]), np.cos(hh[i]), 0], 
                  [0, 0, 1]])
    
    P = np.array([[np.cos(pp[i]), -np.sin(pp[i])*np.sin(rr[i]), -np.cos(rr[i])*np.sin(pp[i])], # Makes the tilt matrix for row i
                  [0, np.cos(rr[i]), -np.sin(rr[i])], 
                  [np.sin(pp[i]), np.sin(rr[i])*np.cos(pp[i]), np.cos(pp[i])*np.cos(rr[i])]])
    
    R = H*P*T #Product of transformation, heading, and tilt matrix creates conversion matrix R

    # Retrieves ENU velocities from row i to be converted by R[i]
    enu = np.array([adv1_flagged['Velocity_East(m/s)'][i], adv1_flagged['Velocity_North(m/s)'][i], adv1_flagged['Velocity_Up(m/s)'][i]])
    
    beam1[i] = np.dot(np.linalg.inv(R),enu)[0]
    beam2[i] = np.dot(np.linalg.inv(R),enu)[1]
    beam3[i] = np.dot(np.linalg.inv(R),enu)[2]

### Calculate v_amb
 - v_amb = VR * 2
 - VR = c/(4 * f * tlag(s))
 - c = 1530 (speed of sound measured by instrument)
 - f = instrument frequency (6000kHz)
 - tlag (for vector) = 50/480000 (50 is from system 38 in .hdr file, which indicates nominal velocity of 1m/s)

In [ ]:
c = 1530
f = 6000000
tlag = 50/480000
VR = c / (4 * f * tlag)
v_amb = VR *2
v_amb